In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_validate # <-- ON CHANGE L'IMPORT
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, accuracy_score # <-- ON IMPORTE LES SCORES
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

# df_ml est ton DataFrame de la cellule précédente
if 'df_ml' not in locals():
    print("ERREUR: 'df_ml' n'est pas chargé. Exécute la cellule 'Création de la Cohorte' d'abord.")
    raise NameError("df_ml is not defined")
else:
    print(f"DataFrame 'df_ml' (Cohorte 2018) chargé. Shape: {df_ml.shape}")


# --- 1. DÉFINITION DES FEATURES (ITÉRATION J : "DÉMO MONSTRUEUSE") ---
print("Définition des features pour le Modèle A+ (Démo Étendue)...")

# (On garde la liste des 9 features qui marchent)
DEMO_FEATURES_PLUS = [
    "categorieJuridiqueUniteLegale",
    "trancheEffectifsUniteLegale",
    "activitePrincipaleUniteLegale",
    "categorieEntreprise",
    "economieSocialeSolidaireUniteLegale",
    # "societeMissionUniteLegale", # On l'a oubliée dans le script 01, on la laisse de côté
    "moisCreation",
    "departement",
    "trancheEffectifsSiege",
    "caractereEmployeurSiege"
]

TARGET = "is_failed_in_3y"

X = df_ml.select(DEMO_FEATURES_PLUS).fill_null("INCONNU").to_pandas()
y = df_ml.select(TARGET).to_pandas().squeeze()

print(f"Features (X) sélectionnées: {X.columns.to_list()}")
print(f"Target (y) sélectionnée: {y.name}")

# --- 2. GESTION DU DÉSÉQUILIBRE ---
scale_pos_weight = y.value_counts()[0] / y.value_counts()[1]
print(f"Ratio de déséquilibre : {scale_pos_weight:.2f}")

# --- 3. PRÉPARATION (Preprocessing) ---
print("Preprocessing avec OneHotEncoder...")
categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
preprocessor = ColumnTransformer(
    transformers=[("cat", categorical_transformer, DEMO_FEATURES_PLUS)],
    remainder="passthrough"
)

# --- 4. CRÉATION DE LA PIPELINE DE MODÉLISATION ---
print("Création de la pipeline (Preprocessor + XGBoost)...")
model_A_plus = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        scale_pos_weight=scale_pos_weight, 
        use_label_encoder=False, 
        eval_metric='logloss',
        random_state=42,
        enable_categorical=False 
    ))
])

# --- 5. ENTRAÎNEMENT & ÉVALUATION (CROSS-VALIDATION "MONSTRUEUSE") ---
# ▼▼▼ ON CHANGE TOUTE CETTE PARTIE ▼▼▼
print("Lancement de la K-Fold Cross-Validation (k=5) (AUC, Accuracy, F1)...")
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# On définit les 3 scores qu'on veut
scoring_metrics = {
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'f1_macro': make_scorer(f1_score, average='macro') # On prend f1_macro pour le déséquilibre
}

# On utilise 'cross_validate' (pas 'cross_val_score')
scores = cross_validate(
    model_A_plus, 
    X, y, 
    cv=kfold, 
    scoring=scoring_metrics, 
    n_jobs=-1
)
# ▲▲▲ ON CHANGE TOUTE CETTE PARTIE ▲▲▲

# --- 6. RÉSULTATS (PLUS COMPLETS) ---
print("---")
print("--- RÉSULTATS DU MODÈLE A+ (ITÉRATION 'DÉMO MONSTRUEUSE') ---")
print(f"Score ROC-AUC MOYEN (CV) : {np.mean(scores['test_roc_auc']):.4f} (+/- {np.std(scores['test_roc_auc']):.4f})")
print(f"Score Accuracy MOYEN (CV) : {np.mean(scores['test_accuracy']):.4f} (+/- {np.std(scores['test_accuracy']):.4f})")
print(f"Score F1-Macro MOYEN (CV) : {np.mean(scores['test_f1_macro']):.4f} (+/- {np.std(scores['test_f1_macro']):.4f})")
print("---")
print(f"Score Baseline (Modèle A, 4 features): 0.7619 AUC, 0.40 F1-Macro")
print(f"Score actuel (Modèle A+, {len(DEMO_FEATURES_PLUS)} features): {np.mean(scores['test_roc_auc']):.4f} AUC, {np.mean(scores['test_f1_macro']):.4f} F1-Macro")
print("---")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, RandomizedSearchCV # <-- On change
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
# ▼▼▼ ON IMPORTE SMOTE ET LA PIPELINE IMBLEARN ▼▼▼
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
# ▲▲▲ ON IMPORTE SMOTE ▲▲▲
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, accuracy_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import mlflow # <-- On importe MLFlow
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

# df_ml est ton DataFrame de la cellule précédente
print(f"DataFrame 'df_ml' (Cohorte 2018) chargé. Shape: {df_ml.shape}")

# --- 1. DÉFINITION DES FEATURES (On garde les 9 'Élite') ---
DEMO_FEATURES_PLUS = [
    "categorieJuridiqueUniteLegale",
    "trancheEffectifsUniteLegale",
    "activitePrincipaleUniteLegale",
    "categorieEntreprise",
    "economieSocialeSolidaireUniteLegale",
    "moisCreation",
    "departement",
    "trancheEffectifsSiege",
    "caractereEmployeurSiege"
]
TARGET = "is_failed_in_3y"

X = df_ml.select(DEMO_FEATURES_PLUS).fill_null("INCONNU").to_pandas()
y = df_ml.select(TARGET).to_pandas().squeeze()
print(f"Features (X) sélectionnées: {len(DEMO_FEATURES_PLUS)}")

# --- 2. PRÉPARATION (Preprocessing) ---
print("Preprocessing avec OneHotEncoder...")
categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse_output=False, min_frequency=0.001) # On ignore les catégories trop rares
preprocessor = ColumnTransformer(
    transformers=[("cat", categorical_transformer, DEMO_FEATURES_PLUS)],
    remainder="passthrough"
)

# --- 3. CRÉATION DE LA PIPELINE "MONSTRUEUSE" (AVEC SMOTE) ---
print("Création de la pipeline (Preprocessor + SMOTE + XGBoost)...")
model_A_plus = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)), # <-- L'ITÉRATION MAGIQUE
    ('classifier', XGBClassifier( 
        # On n'a plus besoin de scale_pos_weight
        use_label_encoder=False, 
        eval_metric='logloss',
        random_state=42,
        enable_categorical=False 
    ))
])

# --- 4. DÉFINIR LA GRILLE DE TUNING ---
# On va tester 10 combinaisons au hasard
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [3, 5, 7, 10],
    'classifier__learning_rate': [0.1, 0.05, 0.01],
    'classifier__subsample': [0.7, 1.0]
}

# --- 5. ENTRAÎNEMENT & ÉVALUATION (RANDOMIZED SEARCH) ---
print("Lancement du 'Monstrous' Tuning (RandomizedSearchCV)...")
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# On optimise pour le F1-Score ! (C'est notre métrique "business")
random_search = RandomizedSearchCV(
    estimator=model_A_plus,
    param_distributions=param_grid,
    n_iter=10, # 10 essais (rapide, mais tu peux monter à 20-30)
    cv=kfold,
    scoring='f1_macro', # <-- ON OPTIMISE POUR LE F1
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# --- 6. CONFIGURER MLFLOW ---
mlflow.set_experiment("Projet_SIRENE_Classification_Finale")
mlflow.sklearn.autolog() # Va logger tous les essais du RandomizedSearch

print("Lancement de l'entraînement (peut prendre 10-20 minutes)...")
with mlflow.start_run() as run:
    random_search.fit(X, y)
    mlflow.log_param("model_type", "Model_A_plus_SMOTE_Tuned")
    mlflow.log_metric("best_f1_macro", random_search.best_score_)

# --- 7. RÉSULTATS (PLUS COMPLETS) ---
print("---")
print("--- RÉSULTATS DU MODÈLE A++ (SMOTE + TUNING) ---")
print(f"Meilleur Score F1-Macro (CV) : {random_search.best_score_:.4f}")
print("Meilleurs Hyperparamètres :")
print(random_search.best_params_)
print("---")
print(f"Score Baseline (Modèle A, 4 feats): 0.40 F1-Macro")
print(f"Score (Modèle A+, 9 feats): 0.41 F1-Macro")
print(f"Score actuel (Modèle A++, SMOTE+Tuned): {random_search.best_score_:.4f} F1-Macro")
print("---")
print("Lance 'mlflow ui' dans ton terminal pour voir le dashboard de tous les essais !")

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import make_scorer, f1_score
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import mlflow
import warnings
import os

warnings.filterwarnings('ignore', category=UserWarning)

# --- 1. CHARGER ET PRÉPARER LES DONNÉES (CORRIGÉ) ---
print("Chargement et préparation de 'df_ml'...")
# ▼▼▼ ON UTILISE LE BON FICHIER "CLEAN" ▼▼▼
PATH_MASTER_DEMO = "../Data/processed/sirene_infos_CLEAN.parquet"
# ▲▲▲ ON UTILISE LE BON FICHIER "CLEAN" ▲▲▲

try:
    df_master = pl.read_parquet(PATH_MASTER_DEMO)
    print(f"Fichier 'sirene_infos_CLEAN.parquet' chargé. Shape: {df_master.shape}")
except FileNotFoundError:
    print(f"ERREUR FATALE: Fichier non trouvé : {PATH_MASTER_DEMO}")
    print("Relance ton notebook de Data Prep pour le créer.")
    raise

# On filtre la Cohorte 2018
df_ml = df_master.filter(
    pl.col("dateCreationUniteLegale").dt.year() == 2018
).with_columns(
    # On crée la Target
    (pl.col("dateCreationUniteLegale").dt.offset_by("3y")).alias("date_limite_3_ans")
).with_columns(
    pl.when(
        (pl.col("dateFermeture").is_not_null()) & 
        (pl.col("dateFermeture") < pl.col("date_limite_3_ans"))
    ).then(1)
    .otherwise(0)
    .alias("is_failed_in_3y")
)

# --- 2. DÉFINITION DES FEATURES (Les 9 features "Démo") ---
# (On prend toutes les features qu'on a créées dans le script 01)
DEMO_FEATURES_PLUS = [
    "categorieJuridiqueUniteLegale",
    "trancheEffectifsUniteLegale",
    "activitePrincipaleUniteLegale",
    "categorieEntreprise",
    "economieSocialeSolidaireUniteLegale",
    # "societeMissionUniteLegale", # On l'a oubliée dans le script 01
    "departement",
    pl.col("dateCreationUniteLegale").dt.month().alias("moisCreation"),
    "trancheEffectifsSiege",
    "caractereEmployeurSiege"
]
TARGET = "is_failed_in_3y"

# On doit extraire les noms (strings) pour le preprocessor
DEMO_FEATURES_NAMES = [
    f.meta.output_name() if isinstance(f, pl.Expr) else f for f in DEMO_FEATURES_PLUS
]

X = df_ml.select(DEMO_FEATURES_PLUS).fill_null("INCONNU").to_pandas()
y = df_ml.select(TARGET).to_pandas().squeeze()
print(f"Features (X) sélectionnées: {len(DEMO_FEATURES_NAMES)}")

# --- 3. PRÉPARATION (OrdinalEncoder) ---
print("Preprocessing avec OrdinalEncoder...")
categorical_features = DEMO_FEATURES_NAMES
categorical_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1) 

preprocessor = ColumnTransformer(
    transformers=[("cat", categorical_transformer, categorical_features)],
    remainder="passthrough"
)

# --- 4. CRÉATION DE LA PIPELINE "LGBM" (AVEC SMOTE) ---
print("Création de la pipeline (OrdinalEncoder + SMOTE + LightGBM)...")
categorical_feature_indices = [X.columns.get_loc(c) for c in categorical_features]

model_LGBM = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)), 
    ('classifier', LGBMClassifier( 
        random_state=42,
        n_jobs=-1,
        categorical_feature=categorical_feature_indices
    ))
])

# --- 5. DÉFINIR LA GRILLE DE TUNING (pour LGBM) ---
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [5, 7, 10, -1],
    'classifier__learning_rate': [0.1, 0.05, 0.01],
    'classifier__subsample': [0.7, 1.0],
    'classifier__num_leaves': [31, 50, 70]
}

# --- 6. ENTRAÎNEMENT & ÉVALUATION (RANDOMIZED SEARCH) ---
print("Lancement du 'LGBM' Tuning (RandomizedSearchCV)...")
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=model_LGBM,
    param_distributions=param_grid,
    n_iter=10,
    cv=kfold,
    scoring='f1_macro',
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# --- 7. CONFIGURER MLFLOW ---
mlflow.set_experiment("Projet_SIRENE_Classification_LGBM")
mlflow.sklearn.autolog() 

print("Lancement de l'entraînement...")
with mlflow.start_run() as run:
    random_search.fit(X, y)
    mlflow.log_param("model_type", "Model_LGBM_SMOTE_Tuned")
    mlflow.log_metric("best_f1_macro", random_search.best_score_)

# --- 8. RÉSULTATS (PLUS COMPLETS) ---
print("---")
print("--- RÉSULTATS DU MODÈLE LGBM (SMOTE + TUNING) ---")
print(f"Meilleur Score F1-Macro (CV) : {random_search.best_score_:.4f}")
print("Meilleurs Hyperparamètres :")
print(random_search.best_params_)
print("---")
print(f"Score (Modèle A++, XGB+OHE): 0.4976 F1-Macro")
print(f"Score actuel (Modèle LGBM+Ordinal): {random_search.best_score_:.4f} F1-Macro")
print("---")
print("Lance 'mlflow ui' dans ton terminal pour voir le dashboard de tous les essais !")

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, RandomizedSearchCV # <-- On garde RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import make_scorer, roc_auc_score, f1_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import mlflow
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

# --- 1. CHARGER LE DATASET ML FINAL (LE COMPLET) ---
print("Chargement du dataset ML 'CLEAN'...")
PATH_ML_CLEAN = "../Data/processed/sirene_infos_CLEAN.parquet"

try:
    df_master = pl.read_parquet(PATH_ML_CLEAN)
    print(f"Dataset chargé. Shape: {df_master.shape}")
except Exception as e:
    print(f"ERREUR: Fichier '{PATH_ML_CLEAN}' non trouvé.")
    raise e

# --- 2. FILTRER LA COHORTE 2018 ET CRÉER LA TARGET ---
print("Filtrage Cohorte 2018 et création de la Target...")
df_ml = df_master.filter(
    pl.col("dateCreationUniteLegale").dt.year() == 2018
).with_columns(
    (pl.col("dateCreationUniteLegale").dt.offset_by("3y")).alias("date_limite_3_ans")
).with_columns(
    pl.when(
        (pl.col("dateFermeture").is_not_null()) & 
        (pl.col("dateFermeture") < pl.col("date_limite_3_ans"))
    ).then(1)
    .otherwise(0)
    .alias("is_failed_in_3y")
)

# --- 3. DÉFINITION DES FEATURES (Les 9 "Élite") ---
DEMO_FEATURES_PLUS = [
    "categorieJuridiqueUniteLegale",
    "trancheEffectifsUniteLegale",
    "activitePrincipaleUniteLegale",
    "categorieEntreprise",
    "economieSocialeSolidaireUniteLegale",
    "moisCreation",
    "departement",
    "trancheEffectifsSiege",
    "caractereEmployeurSiege"
]
TARGET = "is_failed_in_3y"

X = df_ml.select(DEMO_FEATURES_PLUS).fill_null("INCONNU").to_pandas()
y = df_ml.select(TARGET).to_pandas().squeeze()
print(f"Features (X) sélectionnées: {len(DEMO_FEATURES_PLUS)}")

# --- 4. PRÉPARATION (Preprocessing - OHE) ---
# On a prouvé que OHE + XGBoost marchait mieux
print("Preprocessing avec OneHotEncoder...")
categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse_output=False, min_frequency=0.01) # min_frequency aide à réduire le bruit
preprocessor = ColumnTransformer(
    transformers=[("cat", categorical_transformer, DEMO_FEATURES_PLUS)],
    remainder="passthrough"
)

# --- 5. CRÉATION DE LA PIPELINE (OHE + SMOTE + XGBoost) ---
print("Création de la pipeline (OHE + SMOTE + XGB Classifier)...")

# On utilise ImbPipeline pour que SMOTE ne s'applique qu'au train set dans la CV
model_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)), 
    ('classifier', XGBClassifier( 
        use_label_encoder=False, 
        eval_metric='logloss',
        random_state=42
    ))
])

# --- 6. DÉFINIR LA GRILLE DE TUNING (pour XGBoost) ---
# On va tester des paramètres différents
param_grid = {
    'classifier__n_estimators': [100, 200, 400], # Nb d'arbres
    'classifier__max_depth': [3, 5, 8],          # Profondeur
    'classifier__learning_rate': [0.05, 0.1, 0.2],
    'classifier__subsample': [0.7, 0.8, 1.0],
    'classifier__colsample_bytree': [0.7, 0.8, 1.0] # % de features utilisées par arbre
}

# --- 7. ENTRAÎNEMENT & ÉVALUATION (RANDOMIZED SEARCH) ---
print("Lancement du 'XGBoost Tuning' (RandomizedSearchCV)...")
kfold = KFold(n_splits=3, shuffle=True, random_state=42) # On met k=3 pour aller plus vite

# On teste 10 combinaisons au hasard
random_search = RandomizedSearchCV(
    estimator=model_pipeline,
    param_distributions=param_grid,
    n_iter=10, 
    cv=kfold,
    scoring='f1_macro', # On optimise pour le F1
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# --- 8. CONFIGURER MLFLOW ---
mlflow.set_experiment("Projet_SIRENE_Classification_XGB_Tuned")
mlflow.sklearn.autolog() 

print("Lancement de l'entraînement (peut prendre 10-20 minutes)...")
with mlflow.start_run() as run:
    random_search.fit(X, y)
    mlflow.log_param("model_type", "Model_A_plus_plus_XGB_SMOTE_Tuned")
    mlflow.log_metric("best_f1_macro", random_search.best_score_)

# --- 9. RÉSULTATS (PLUS COMPLETS) ---
print("---")
print("--- RÉSULTATS DU MODÈLE A++ (XGBoost + SMOTE + TUNING) ---")
print(f"Meilleur Score F1-Macro (CV) : {random_search.best_score_:.4f}")
print("Meilleurs Hyperparamètres :")
print(random_search.best_params_)
print("---")
print(f"Score précédent (Modèle A++, non-tuné): 0.4976 F1-Macro")
print(f"Score actuel (Modèle A++, Tuné): {random_search.best_score_:.4f} F1-Macro")
print("---")
print("Lance 'mlflow ui' dans ton terminal pour voir le dashboard de tous les essais !")